<a href="https://colab.research.google.com/github/m-shilpa/Transformer_Memory_As_A_Differentiable_Search_Index/blob/main/IR_NQ_dataset_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing of the NQ Dataset


The dataset used here is the NQ Simplified train set. The dataset can be download from the below link

https://ai.google.com/research/NaturalQuestions/download

In [ ]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
%%capture
!pip install transformers[sentencepiece]
!pip install nlp
!pip install pytorch-lightning
!pip install wandb

In [ ]:
import os
import string
import pandas as pd
import numpy as np
import random
import re
from datetime import datetime
import gzip
import json

pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
dataset_gz_path = '/content/gdrive/MyDrive/IR/v1.0-simplified_simplified-nq-train.jsonl.gz'

In [ ]:
N = 50000

def load_data(dataset_gz_path, N,save_csv=False):
    print(datetime.now())
    with gzip.open(dataset_gz_path, 'r') as f_in:
        df = pd.json_normalize([json.loads(next(f_in).decode('utf-8')) for x in range(N)])

    print(datetime.now())
    if save_csv:
        df.to_csv(f'data_{str(N)}.csv')
        print(f'File saved to data_{str(N)}.csv')
    return df

In [ ]:
def get_data_size(dataset_gz_path):

    i=0
    with gzip.open(dataset_gz_path, 'r') as f_in:
        while True:
            try:
                next(f_in)
                i=i+1
            except:
                break
    print('No. of lines in file is', i)

get_data_size(dataset_gz_path) 

No. of lines in file is 307373


In [ ]:
load_data(dataset_gz_path, N=50000,save_csv=True)

In [ ]:
df = pd.read_csv('data_50000.csv')

In [ ]:
df['question_text']

0        which is the most common use of opt-in e-mail marketing                       
1        how i.met your mother who is the mother                                       
2        what type of fertilisation takes place in humans                              
3        who had the most wins in the nfl                                              
4        what happened to the lost settlement of roanoke                               
                              ...                                                      
49995    show that the product of resistance and capacitance (rc) has the units of time
49996    when do avery and juliette get back together                                  
49997    when did the us pay off the revolutionary war debt off                        
49998    come sail away come sail away with me                                         
49999    in what country do you find the kivi                                          
Name: question_text, Length: 500

In [ ]:
len(df)

50000

In [ ]:
print(len(df))
print('Max no. of words/tokens in questions',df['question_text'].str.split().str.len().max())
print('Min no. of words/tokens in questions',df['question_text'].str.split().str.len().min())

50000
Max no. of words/tokens in questions 23
Min no. of words/tokens in questions 3


In [ ]:
df.columns

Index(['Unnamed: 0', 'document_text', 'long_answer_candidates',
       'question_text', 'annotations', 'document_url', 'example_id'],
      dtype='object')

In [ ]:
# as seen below there are html tags in the document text that needs to be removed

df['document_text']

0        Email marketing - Wikipedia <H1> Email marketing </H1> Jump to : navigation , search <Table> <Tr> <Td> </Td> <Td> ( hide ) This article has multiple issues . Please help improve it or discuss these issues on the talk page . ( Learn how and when to remove these template messages ) <Table> <Tr> <Td> </Td> <Td> This article needs additional citations for verification . Please help improve this article by adding citations to reliable sources . Unsourced material may be challenged and removed . ( September 2014 ) ( Learn how and when to remove this template message ) </Td> </Tr> </Table> <Table> <Tr> <Td> </Td> <Td> This article possibly contains original research . Please improve it by verifying the claims made and adding inline citations . Statements consisting only of original research should be removed . ( January 2015 ) ( Learn how and when to remove this template message ) </Td> </Tr> </Table> ( Learn how and when to remove this template message ) </Td> </Tr> </Table> <Table>

In [ ]:
df['document_text'].iloc[0]

"Email marketing - Wikipedia <H1> Email marketing </H1> Jump to : navigation , search <Table> <Tr> <Td> </Td> <Td> ( hide ) This article has multiple issues . Please help improve it or discuss these issues on the talk page . ( Learn how and when to remove these template messages ) <Table> <Tr> <Td> </Td> <Td> This article needs additional citations for verification . Please help improve this article by adding citations to reliable sources . Unsourced material may be challenged and removed . ( September 2014 ) ( Learn how and when to remove this template message ) </Td> </Tr> </Table> <Table> <Tr> <Td> </Td> <Td> This article possibly contains original research . Please improve it by verifying the claims made and adding inline citations . Statements consisting only of original research should be removed . ( January 2015 ) ( Learn how and when to remove this template message ) </Td> </Tr> </Table> ( Learn how and when to remove this template message ) </Td> </Tr> </Table> <Table> <Tr> <T

In [ ]:
from bs4 import BeautifulSoup
def remove_html(doc):
    return BeautifulSoup(doc).text

In [ ]:
from bs4 import BeautifulSoup
def remove_html(doc):
    return BeautifulSoup(doc).text

def save_processed_data_to_csv(dataset_gz_path, N,save_path=None, num_words=1000):
    
    if save_path ==None:
        save_path = 'data_clipped_documents.csv'
        
    print(datetime.now())
    with gzip.open(dataset_gz_path, 'r') as f_in:
        for x in range(N):
            if x==0:
                header = True
            else:
                header = False
            df = pd.json_normalize([json.loads(next(f_in).decode('utf-8')) ])
            df['document_text'] = df['document_text'].apply(remove_html).str.split().str[:num_words].apply(' '.join)
            df[['document_text', 
                  'question_text', 'document_url']].to_csv(save_path,
                                                           mode='a',index=False, header=header)    

    print(datetime.now())
    print(f'File saved to {save_path}')


In [ ]:
save_processed_data_to_csv(dataset_gz_path, N=100000)

2022-04-10 14:27:34.409798
2022-04-10 15:31:45.702083
File saved to data_clipped_documents.csv


In [ ]:
df = pd.read_csv('data_clipped_documents.csv')

In [ ]:
df = df.reset_index()

In [ ]:
df.rename(columns={'index':'doc_id'},inplace=True)

In [ ]:
df.columns

Index(['doc_id', 'document_text', 'question_text', 'document_url'], dtype='object')

In [ ]:
df.to_csv('/content/gdrive/MyDrive/IR/data_100K_clipped_documents_1k_words.csv', index=False)

In [ ]:
df.head()

doc_id  \
0  0        
1  1        
2  2        
3  3        
4  4        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    